In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
from PIL import Image
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from transformers import (
    CLIPProcessor, CLIPTokenizer,
    CLIPVisionModelWithProjection, CLIPTextModelWithProjection,
    get_scheduler, AutoTokenizer, AutoProcessor
)
from torchmetrics.classification import Accuracy, AUROC

In [ ]:
%cd /content/drive/MyDrive/大專生計畫/data

/content/drive/MyDrive/大專生計畫/data


In [ ]:
#prompt_template = "The meme content is [MASK]"
prompt_template = "This is [MASK]"
adjectives = ["normal", "hate"]
#adjectives = ["good", "bad"]
# adjectives = ["normal", "bad"]
#adjectives = ["good", "hate"]

In [ ]:
CLIP_MODEL = "openai/clip-vit-large-patch14"
tokenizer = AutoTokenizer.from_pretrained(CLIP_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
text_encoder = CLIPTextModelWithProjection.from_pretrained(CLIP_MODEL)
tokenizer = AutoTokenizer.from_pretrained(CLIP_MODEL)
visual_encoder = CLIPVisionModelWithProjection.from_pretrained(CLIP_MODEL)
processor = AutoProcessor.from_pretrained(CLIP_MODEL)

In [ ]:
def encode_sentences(sentences, model, tokenizer, device):
    inputs = tokenizer(sentences, padding=True, return_tensors="pt")
    inputs.to(device)
    model.to(device)
    outputs = model(**inputs)
    return outputs.text_embeds

def encode_image(url, model, processor, device):
    image = Image.open(url)
    inputs = processor(images=image, return_tensors="pt")
    inputs.to(device)
    model.to(device)
    outputs = model(**inputs)
    return outputs.image_embeds

def classify(image_embeds, text_embeds):
    result = torch.matmul(text_embeds, image_embeds.T)
    result = result.detach().cpu()
    return torch.softmax(result, dim=-1)

def construct_prompt(text):
    sentences = []
    for adj in adjectives:
        classify_prompt = prompt_template.replace("[MASK]", adj)
        sentences.append(" ".join([text, classify_prompt]))
    return sentences

In [ ]:
train_path = "processed_train.jsonl"
dev_path = "processed_dev.jsonl"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
train_df = pd.read_json(train_path, lines=True)
dev_df = pd.read_json(dev_path, lines=True)
dev_df.head()

,id,img,label,text,caption
0,8291,img/08291.png,1,white people is this a shooting range,two children sitting on the ground eating fruit
1,46971,img/46971.png,1,bravery at its finest,a man with a beard and a shirt on
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,a woman sitting at a desk with a laptop
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,a person holding a bottle of beer
4,80243,img/80243.png,1,mississippi wind chime,a black and white photo of a group of men walk...


In [ ]:
error_ids = []
answers = []
predicts = []
for _, row in tqdm(dev_df.iterrows()):
    with torch.no_grad():
        sentences = construct_prompt("")
        url = row["img"]
        text_embeds = encode_image(url, visual_encoder, processor, device)
        image_embeds = encode_sentences(sentences, text_encoder, tokenizer, device)
        pred = classify(image_embeds, text_embeds)
        ans = row["label"]
        if torch.argmax(pred).item() != ans:
            # collect error cases
            error_ids.append(row["id"])
        # collect the result
        predicts.append(pred)
        answers.append(ans)

0it [00:00, ?it/s]